## Testing Access of CTA Bus Tracker

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



### Pulls
Limit of 100,000 pulls/day/key. There are 107 bus routes. We can query up to 10 routes at once.

### Features

- vid:
- tms
- ...

In [2]:
# imports

import requests

In [3]:
# liv's api key: xYZ6rHMweaTy5Ejzahk5Y4eQs

url_bus = 'https://www.ctabustracker.com/bustime/api/v3/?key=xYZ6rHMweaTy5Ejzahk5Y4eQs'
url_predictions = 'https://www.ctabustracker.com/bustime/api/v3/getvehicles?key=xYZ6rHMweaTy5Ejzahk5Y4eQs&rt=201,55&format=json'

r = requests.get(url_predictions)


# print(r.text)



In [4]:
bus = r.json()

In [5]:
bus

{'bustime-response': {'vehicle': [{'vid': '1570',
    'tmstmp': '20260218 17:31',
    'lat': '41.793087005615234',
    'lon': '-87.73565673828125',
    'hdg': '90',
    'pid': 5424,
    'rt': '55',
    'des': 'Museum of Science & Industry',
    'pdist': 5991,
    'dly': False,
    'tatripid': '204604',
    'origtatripno': '273985436',
    'tablockid': '55 -651',
    'zone': '',
    'mode': 1,
    'psgld': 'N/A',
    'stst': 61710,
    'stsd': '2026-02-18'},
   {'vid': '8057',
    'tmstmp': '20260218 17:30',
    'lat': '41.79317855834961',
    'lon': '-87.72671055793762',
    'hdg': '85',
    'pid': 5424,
    'rt': '55',
    'des': 'Museum of Science & Industry',
    'pdist': 8426,
    'dly': False,
    'tatripid': '204605',
    'origtatripno': '273985437',
    'tablockid': '55 -659',
    'zone': '',
    'mode': 1,
    'psgld': 'N/A',
    'stst': 62280,
    'stsd': '2026-02-18'},
   {'vid': '8019',
    'tmstmp': '20260218 17:30',
    'lat': '41.79350357055664',
    'lon': '-87.705194091

In [ ]:
# import requests
# import pandas as pd
# import time
# from datetime import datetime
# from zoneinfo import ZoneInfo

# ####### Main Functions - Pedro's Version #######

# ##############################
# # FUNCTION 1 - API EXTRACTING
# ##############################

# def get_api(url):
#   r = requests.get(url_predictions)

#   if r.ok:
#     data = r.json()
#     print('API Request complete')
#     return data

#   else:
#     raise ValueError('Could not complete API Request')

# ##############################
# # FUNCTION 2 - SAVING DATA
# ##############################

# def save_data(data):
#   try:
#     vehicles = data['bustime-response']['vehicle']
#     df = pd.DataFrame(vehicles)

#     chicago_tz = ZoneInfo("America/Chicago")
#     timestamp = datetime.now(chicago_tz).strftime("%Y-%m-%d_%H-%M-%S")
#     filename = f"{timestamp}_bus_data.csv"

#     df.to_csv(filename, index=False)
#     print(f'Data saved to {filename}')

#     return filename, len(df)

#   except:
#     raise ValueError('No active vehicle data')


# ##############################
# # FUNCTION 3 - MAIN RUN
# ##############################
# def main(url, sleep_time=30, runtime_seconds=3600):
#   n_calls = 0
#   try:
#     start_time = time.time()
#     end_time = start_time + runtime_seconds

#     while time.time() - start_time < end_time:
#       data = get_api(url)
#       filename, n_rows = save_data(data)
#       n_calls += 1
#       print(f'{n_rows} rows were saved to {filename}')

#       time.sleep(sleep_time)

#   except Exception as e:
#       print(f"[{n_calls+1}] Error: {e}")


In [6]:
import os
import requests
import pandas as pd
import time
from datetime import datetime
from zoneinfo import ZoneInfo

CHICAGO_TZ = ZoneInfo("America/Chicago")

def chunk_list(xs, n=10):
    return [xs[i:i+n] for i in range(0, len(xs), n)]

def get_routes(api_key):
    print("Getting routes...")
    url = f"https://www.ctabustracker.com/bustime/api/v3/getroutes?key={api_key}&format=json"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()

    routes = data.get("bustime-response", {}).get("routes", [])
    if not routes:
        err = data.get("bustime-response", {}).get("error", [])
        raise ValueError(f"No routes returned. Error: {err}")

    return [rt["rt"] for rt in routes if "rt" in rt]

def get_api(url):
    r = requests.get(url, timeout=30)
    if r.ok:
        return r.json()
    raise ValueError(f"API request failed (status={r.status_code})")

def append_vehicles_to_csv(data, outfile, pulled_at, rt_chunk):
    vehicles = data.get("bustime-response", {}).get("vehicle", None)
    if not vehicles:
        # no vehicles is normal sometimes; don't crash the whole run
        return 0

    df = pd.DataFrame(vehicles)

    # add metadata columns so you can trace pulls later
    df["pulled_at"] = pulled_at
    df["rt_chunk"] = rt_chunk

    file_exists = os.path.exists(outfile)
    df.to_csv(outfile, mode="a", header=not file_exists, index=False)
    return len(df)

def main(api_key, per_chunk_sleep=5, per_sweep_sleep=30, runtime_hours=1200, break_hours=300, out_dir="."):
    routes = get_routes(api_key)
    chunks = chunk_list(routes, n=10)

    start_stamp = datetime.now(CHICAGO_TZ).strftime("%Y-%m-%d_%H-%M-%S")
    outfile = os.path.join(out_dir, f"bus_data_{start_stamp}_chicago.csv")

    print(f"Found {len(routes)} routes -> {len(chunks)} chunks")
    print(f"Writing EVERYTHING to one file:\n  {outfile}\n")
    print(f"Chunk sleep: {per_chunk_sleep}s | Sweep sleep: {per_sweep_sleep}s | Runtime: {runtime_hours} hours\n")

    start = time.time()
    end = start + runtime_hours*3600

    sweep_num = 0
    call_num = 0
    total_rows = 0

    while time.time() < end:
        sweep_num += 1
        print(f"--- Sweep {sweep_num} @ {datetime.now(CHICAGO_TZ).strftime('%Y-%m-%d %H:%M:%S %Z')} ---")

        for i, chunk in enumerate(chunks):
            if time.time() >= end:
                break

            rt_param = ",".join(chunk)
            url = (
                "https://www.ctabustracker.com/bustime/api/v3/getvehicles"
                f"?key={api_key}&rt={rt_param}&format=json"
            )

            pulled_at = datetime.now(CHICAGO_TZ).strftime("%Y-%m-%d %H:%M:%S %Z")

            try:
                data = get_api(url)
                n_rows = append_vehicles_to_csv(data, outfile, pulled_at=pulled_at, rt_chunk=rt_param)
                call_num += 1
                total_rows += n_rows
                print(f"[Call {call_num}] chunk {i+1}/{len(chunks)}: appended {n_rows} rows (total {total_rows})")
            except Exception as e:
                call_num += 1
                print(f"[Call {call_num}] chunk {i+1}/{len(chunks)} ERROR: {e}")

            if time.time() < end:
                time.sleep(min(per_chunk_sleep, max(0, end - time.time())))

        if time.time() < end:
            sleep_now = min(per_sweep_sleep, max(0, end - time.time()))
            print(f"--- Sweep {sweep_num} complete. Sleeping {sleep_now:.0f}s ---\n")
            time.sleep(sleep_now)

    print(f"\nDone. Sweeps: {sweep_num}, calls: {call_num}, total rows written: {total_rows}")
    print(f"Output file: {outfile}")
    return outfile


In [ ]:
key = 'NPeYcrgS6Pt432G5F64u8jgQD'
main(key)

# Calculating Delays Percentage

In [ ]:
# Change file manually for now
file = pd.read_csv('bus_data_2026-02-18_17-31-35_chicago.csv')

## Analysis 1: Percentage of Observed Delays

In [ ]:
def delay_percentage(df, route_col="rt", delay_col="dly"):
  '''
  This function analyzes the percentage of time a route is delayed (delayed observations / total observations per route).
  '''

  # Ensuring that delays are read as booleans
  df[delay_col] = df[delay_col].astype(bool)
  
  
  delay = (df.groupby(route_col)[delay_col]
           .agg(delayed_obs="sum", total_obs="count")
           .assign(delay_pct=lambda x: 100 * x["delayed_obs"] / x["total_obs"])
           .reset_index()
           .sort_values("delay_pct", ascending=False))

  return delay

In [17]:
delay_stats = delay_percentage(file)
delay_stats.head()


,rt,delayed_obs,total_obs,delay_pct
1,100,3,12,25.000000
73,59,3,24,12.500000
100,84,2,18,11.111111
3,106,1,10,10.000000
41,26,3,30,10.000000


## Analysis 2: Delay rate per bus; Average per Route

In [ ]:
def delay_per_bus_route(df, route_col="rt", vid_col="vid", delay_col="dly"):
      '''
      This function calculates the delay rate for each vehicle, and then averages the rate across the number of vehicles.
      '''
      
      d = df.copy()
      
      # Ensuring boleean
      d[delay_col] = d[delay_col].astype(bool)
      
      # Creating the average per link
      per_vehicle = (d.groupby([route_col, vid_col])[delay_col]
                     .mean()
                     .reset_index(name="delay_share_vid"))
      
      # Averaging percentages by the number of buses
      out = (per_vehicle.groupby(route_col)["delay_share_vid"]
             .mean()
             .mul(100)
             .reset_index(name="delay_pct_vehicle_weighted")
             .sort_values("delay_pct_vehicle_weighted", ascending=False))
      
      
      return out


In [ ]:
a1 = delay_per_bus_route(file)
a1.head()

,rt,delay_pct_vehicle_weighted
3,106,25.000000
1,100,25.000000
73,59,12.500000
100,84,11.111111
41,26,10.000000


## Analysis 3: Share of vehicles delayed by Route at each Sweep

In [ ]:
def delay_vehicles_delayed(df, route_col="rt", timestamp_col="tmstmp", vid_col="vid", delay_col="dly"):
       
       '''
       This function calculates the percentage of vehicles delayed in a route at each timestamp
       '''
       
       d = df.copy()
       
       # Ensuring delay is read as boolean
       d[delay_col] = d[delay_col].astype(bool)
       
       # Dropping duplicates if no new observations
       d = d.drop_duplicates([route_col, timestamp_col, vid_col])
       
       per_pull = (d.groupby([route_col, timestamp_col])[delay_col]
                   .mean()
                   .reset_index(name="delay_share_tmstmp"))
       
       out = (per_pull.groupby(route_col)["delay_share_tmstmp"]
              .mean()
              .mul(100)
              .reset_index(name="delay_pct_tmptmp_weighted")
              .sort_values("delay_pct_pull_weighted", ascending=False))
       
       return out


In [ ]:
a2 = delay_vehicles_delayed(file, route_col="rt", vid_col="vid", delay_col="dly")

a2.head()

,rt,delay_pct_pull_weighted
1,100,25.000000
73,59,12.500000
100,84,11.111111
41,26,10.000000
3,106,8.333333


# Initial Logic

In [37]:
import numpy as np
import pandas as pd

# -----------------------------
# Helpers
# -----------------------------
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1-a))

def gtfs_times_for_date(arrival_times_series, service_date):
    """
    arrival_times_series: strings HH:MM:SS (HH can exceed 24)
    returns pd.DatetimeIndex aligned to service_date
    """
    t = arrival_times_series.astype(str).str.split(":", expand=True)
    hh = pd.to_numeric(t[0], errors="coerce")
    mm = pd.to_numeric(t[1], errors="coerce")
    ss = pd.to_numeric(t[2], errors="coerce")
    base = pd.to_datetime(service_date).normalize()
    return base + pd.to_timedelta(hh, unit="h") + pd.to_timedelta(mm, unit="m") + pd.to_timedelta(ss, unit="s")

def first_entry_time(pings_sorted, stop_lat, stop_lon, tol_m=24):
    """
    Returns first time bus enters within tol_m of stop, with interpolation on boundary crossing.
    pings_sorted must have columns ['ts','lat','lon'] and sorted by ts.
    """
    d = haversine_m(
        pings_sorted["lat"].to_numpy(),
        pings_sorted["lon"].to_numpy(),
        stop_lat, stop_lon
    )
    inside = d <= tol_m
    if not inside.any():
        return None

    i = int(np.argmax(inside))
    if i == 0:
        return pings_sorted["ts"].iloc[0]

    if inside[i] and (not inside[i-1]):
        t0 = pings_sorted["ts"].iloc[i-1]
        t1 = pings_sorted["ts"].iloc[i]
        d0 = d[i-1]; d1 = d[i]
        frac = (d0 - tol_m) / (d0 - d1 + 1e-12)
        return t0 + (t1 - t0) * float(frac)

    return pings_sorted["ts"].iloc[i]

# -----------------------------
# 1) Build stop arrival events from AVL (event-based, not ping-based)
# -----------------------------
def build_stop_events(avl_df, stops_df, tol_m=24, ts_col="tmstmp"):
    d = avl_df.copy()
    d["ts"] = pd.to_datetime(d[ts_col], format="%Y%m%d %H:%M", errors="coerce")
    d = d.dropna(subset=["ts","lat","lon","vid","rt"]).copy()
    d["rt"] = d["rt"].astype(str)

    # We'll create events by checking each stop for each (vid, rt) trajectory.
    # For performance, you usually restrict stops to the route's stop set later.
    events = []

    # Pre-make stops arrays for faster looping
    stops_df = stops_df.copy()
    stops_df["stop_id"] = stops_df["stop_id"].astype(str)
    stops = stops_df[["stop_id","stop_lat","stop_lon"]].to_records(index=False)

    for (vid, rt), grp in d.groupby(["vid","rt"]):
        pings = grp.sort_values("ts")[["ts","lat","lon"]].reset_index(drop=True)

        # Service date for these events
        service_date = pings["ts"].iloc[0].normalize()

        # OPTIONAL: if you have too many stops, you'll want to limit to route stops
        for stop_id, stop_lat, stop_lon in stops:
            arr = first_entry_time(pings, stop_lat, stop_lon, tol_m=tol_m)
            if arr is None:
                continue
            events.append({
                "vid": vid,
                "rt": rt,
                "stop_id": str(stop_id),
                "obs_ts": arr,
                "service_date": service_date
            })

    return pd.DataFrame(events)

# -----------------------------
# 2) Build schedule table per route+stop (using trips.txt to get route_id)
# -----------------------------
def build_route_stop_schedule(stop_times_df, trips_df):
    st = stop_times_df.copy()
    st["trip_id"] = st["trip_id"].astype(str).str.strip()
    st["stop_id"] = st["stop_id"].astype(str).str.strip()

    trips = trips_df.copy()
    trips["trip_id"] = trips["trip_id"].astype(str).str.strip()
    trips["route_id"] = trips["route_id"].astype(str).str.strip()

    # attach route_id (and direction_id if you want)
    out = st.merge(trips[["trip_id","route_id","direction_id"]], on="trip_id", how="inner")
    return out

# -----------------------------
# 3) Match observed stop event to nearest scheduled time
# -----------------------------
def match_events_to_schedule(events_df, route_stop_sched_df, match_window_sec=120, late_threshold_sec=60):
    """
    For each event (rt, stop_id, obs_ts, service_date),
    find nearest scheduled arrival_time among GTFS trips for that route_id+stop_id.
    """
    ev = events_df.copy()
    ev["rt"] = ev["rt"].astype(str)
    ev["stop_id"] = ev["stop_id"].astype(str)

    sched = route_stop_sched_df.copy()
    sched["route_id"] = sched["route_id"].astype(str)
    sched["stop_id"] = sched["stop_id"].astype(str)

    # Pre-index schedule by (route_id, stop_id) to speed up matching
    idx = {}
    for (route_id, stop_id), g in sched.groupby(["route_id","stop_id"]):
        idx[(route_id, stop_id)] = g["arrival_time"].astype(str).to_numpy()

    rows = []
    for r in ev.itertuples(index=False):
        key = (r.rt, r.stop_id)
        if key not in idx:
            continue

        sched_times = idx[key]
        sched_dt = gtfs_times_for_date(pd.Series(sched_times), r.service_date)

        # nearest scheduled
        deltas = (sched_dt - r.obs_ts).abs().dt.total_seconds()
        j = int(deltas.idxmin())
        if deltas.iloc[j] > match_window_sec:
            continue

        matched_sched = sched_dt.iloc[j]
        delay_sec = (r.obs_ts - matched_sched).total_seconds()
        rows.append({
            "vid": r.vid,
            "rt": r.rt,
            "stop_id": r.stop_id,
            "obs_ts": r.obs_ts,
            "sched_ts": matched_sched,
            "delay_sec": delay_sec,
            "delayed": delay_sec > late_threshold_sec
        })

    return pd.DataFrame(rows)

def delay_pct_by_route_from_events(matched_events_df):
    out = (matched_events_df.groupby("rt")["delayed"]
           .agg(delayed_events="sum", total_events="count")
           .assign(delay_pct=lambda x: 100*x["delayed_events"]/x["total_events"])
           .reset_index()
           .sort_values("delay_pct", ascending=False))
    return out


In [38]:
avl_df = pd.read_csv("bus_data_2026-02-18_17-31-35_chicago.csv")
stop_times_df = pd.read_csv("Datasets/stop_times.txt")
trips_df = pd.read_csv("Datasets/trips.txt")
stops_df = pd.read_csv("Datasets/stops.txt")[["stop_id","stop_lat","stop_lon"]]  # strongly recommended

events_df = build_stop_events(avl_df, stops_df, tol_m=24)
route_stop_sched_df = build_route_stop_schedule(stop_times_df, trips_df)

matched_df = match_events_to_schedule(events_df, route_stop_sched_df,
                                      match_window_sec=120,
                                      late_threshold_sec=60)

route_delay_df = delay_pct_by_route_from_events(matched_df)

print(matched_df.head())
print(route_delay_df.head(20))


    vid  rt stop_id                        obs_ts            sched_ts  \
0   608  66    6613 2026-02-18 17:32:16.845637391 2026-02-18 17:31:15   
1   612  66    6712 2026-02-18 17:32:52.857303338 2026-02-18 17:31:16   
2   612  66   14762 2026-02-18 17:31:00.000000000 2026-02-18 17:29:53   
3  1003   1      71 2026-02-18 17:31:00.000000000 2026-02-18 17:30:02   
4  1005  81    3769 2026-02-18 17:31:00.000000000 2026-02-18 17:32:26   

   delay_sec  delayed  
0  61.845637     True  
1  96.857303     True  
2  67.000000     True  
3  58.000000    False  
4 -86.000000    False  
      rt  delayed_events  total_events   delay_pct
71   62H               2             2  100.000000
1    100               2             2  100.000000
4    108               1             1  100.000000
66    57               1             1  100.000000
27   171               1             1  100.000000
8    112               1             1  100.000000
107   96               1             1  100.000000
49    44 